<a href="https://colab.research.google.com/github/charleslow-cmu/bva-capstone/blob/master/dictionary/example_use_dictionary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Authenticate to GCP
project_id = 'bva-appeal'
processed_bucket_name = "bva-appeal-processed-data"
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

Updated property [core/project].


In [0]:
%%capture
# install Google cloud storage FUSE to mount GCP buckets on colab
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse
!mkdir /bva-data
!gcsfuse bva-appeal-processed-data /bva-data # caution on write and/or delete

In [3]:
import pymongo
import os
from functools import reduce
os.chdir('/bva-data/')
from db_config import *     # Credentials stored here

# connect to mongo database
db = pymongo.MongoClient(host=DB_HOST, port=DB_PORT)[DB_DATABASE]
db.authenticate(DB_USER, DB_PASSWORD)

True

In [0]:
# Utility class to label metadata codes
class Dictionary():
    def __init__(self):
        self.issue_dict = db.code_dictionary.find_one({"name": "issue"})['dict']
        self.issdc_dict = db.code_dictionary.find_one({"name": "issdc"})['dict']
        self.cvdisp_dict = db.code_dictionary.find_one({"name": "cvdisp"})['dict']
        self.issue_levels = ["issprog", "isscode", "isslev1", "isslev2", "isslev3"]

    # Given issue codes, return the relevant label
    def label_issue(self, **kwargs):
        list_conditions = []
        for var in self.issue_levels:
            if var in kwargs:
                list_conditions.append(var)
                list_conditions.append(str(kwargs[var]))
        return reduce(dict.__getitem__, list_conditions + ["label"], self.issue_dict)

    # Given issdc code, return label
    def label_issdc(self, issdc):
        return self.issdc_dict["issdc"][str(issdc)]['label']

    # Given cvdisp code, return label
    def label_cvdisp(self, cvdisp):
        return self.cvdisp_dict["cvdisp"][str(cvdisp)]['label']

# Init
dictionary = Dictionary()

In [12]:
dictionary.label_issue(issprog=2, isscode=12, isslev1=1)

'10% under 38 C.F.R. 3.324'

In [9]:
dictionary.label_cvdisp(1)

'Affirmed'

In [11]:
dictionary.label_issdc(4)

'Denied (Board Code)'